In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    def time_(s): 
        print(time.time() * 1000)
        return time.time() * 1000
    def key_err(s):
        raise Exception("Key does not exist")
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'time': time_,'err': key_err}})
    
def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))
    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

<b>Execution time for 10 entries in map</b>

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    a ← time()
    e.init
    b := 0
    while b < 10 do
      e.insert(b,3)
      b := b + 1
    a ← time()
    write(e.get(4))
    b ← time()
    e.insert(1001,23)
    b ← time()
    write(e.len())
""", 'test1.wat', target = 'wat')
!wat2wasm test1.wat
runpywasm("test1.wasm")

Lookup time = 1617757775317.0266 - 1617757775404.4658 = 87 <br>
Insertion time = 1617757584101.363 - 1617757584440.0674 = 341

<b>Execution time for 20 entries in map</b>

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    a ← time()
    e.init
    b := 0
    while b < 20 do
      e.insert(b,3)
      b := b + 1
    a ← time()
    write(e.get(4))
    b ← time()
    e.insert(1001,23)
    b ← time()
    write(e.len())
""", 'test2.wat', target = 'wat')
!wat2wasm test2.wat
runpywasm("test2.wasm")

Lookup time = 1617757038433.1326 - 1617757038477.5427 = 44  <br>
Insertion time = 1617757038477.5427 - 1617757038655.2637 = 140

<b>Execution time for 100 entries in map</b>

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    a ← time()
    e.init
    b := 0
    while b < 100 do
      e.insert(b,3)
      b := b + 1
    a ← time()
    write(e.get(4))
    b ← time()
    e.insert(1001,23)
    b ← time()
    write(e.len())
""", 'test3.wat', target = 'wat')
!wat2wasm test3.wat
runpywasm("test3.wasm")

Lookup time = 1617757089525.4712 - 1617757089538.5867 = 13  <br>
Insertion time = 1617757089538.5867 - 1617757089610.3218 = 87

<b>Execution time for 1000 entries in map</b>

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    a ← time()
    e.init
    b := 0
    while b < 1000 do
      e.insert(b,3)
      b := b + 1
    a ← time()
    write(e.get(4))
    b ← time()
    e.insert(1001,23)
    b ← time()
    write(e.len())
""", 'test4.wat', target = 'wat')
!wat2wasm test4.wat
runpywasm("test4.wasm")

Lookup time = 1617757530145.7568 - 1617757530149.9011 = 4  <br>
Insertion time = 1617757530149.9011 - 1617757530446.7769 = 296

In [ ]:
#clean up files produces by tests
!rm test*